# Vision Edge-AI와 3D Depth Camera

3D Depth Camera는 공간의 깊이 정보를 캡처하여 3차원 데이터로 변환하는 장치입니다. 이러한 카메라는 다양한 응용 분야에서 사용되며, 컴퓨터 비전, 로봇 공학, 증강현실(AR), 가상현실(VR), 의료 이미지 처리 등에서 중요합니다. 본 강의에서는 3D Depth Camera의 원리, 기술, 응용 사례, 그리고 DeepStream을 활용할 수 있는 방법에 대해 설명합니다.

## 03-2. 3D Depth Camera

### 3D Depth Camera의 정의
3D Depth Camera는 객체나 장면의 깊이 정보를 캡처할 수 있는 장치로, 카메라가 각 픽셀의 거리 값을 캡처하여 3차원 공간에서 물체를 인식하거나 측정할 수 있도록 합니다.

### 3D Depth Camera의 원리
1. **스테레오 비전 (Stereo Vision)**:
    * 두 개의 카메라로 물체를 촬영한 이미지를 비교하여 깊이를 계산합니다.
    * 사람의 시각 시스템과 유사한 방식입니다.

2. **구조광 (Structured Light)**:
    * 특정 패턴의 빛을 물체에 투사하고, 왜곡된 패턴을 분석하여 깊이를 측정합니다.
    * 대표적인 예: Microsoft Kinect

3. **비행시간 (Time of Flight, ToF)**:
    * 빛이 물체에 반사되어 돌아오는 시간을 측정하여 깊이를 계산합니다.
    * 정확하고 빠른 깊이 측정이 가능합니다.

4. **LiDAR (Light Detection and Ranging)**:
    * 레이저를 사용해 주변 환경을 스캔하여 깊이를 측정합니다.
    * 자율주행차에 주로 사용됩니다.

### 응용 분야
1. **산업 및 제조**:
    * 로봇 비전: 로봇의 정확한 동작과 위치 결정을 지원.
    * 품질 검사: 제품의 결함 및 크기 측정.
2. **의료**:
    * 3D 스캔: 인체를 스캔하여 맞춤형 의료 장비 제작.
    * 수술 보조: 정확한 수술 계획을 위한 3D 모델 생성.
3. **엔터테인먼트**:
    * 증강현실/가상현실(AR/VR): 몰입형 환경 구축.
    * 게임: 사용자 동작을 추적하여 인터랙션 제공.
4. **도매(Retail)**:
    * 고객 행동 분석: 매장 내 고객의 동선을 추적하고 행동을 분석.
    * 재고 관리: 제품 크기 및 위치를 자동으로 스캔하여 재고를 효율적으로 관리.
    * 무인 매장: 3D 데이터를 활용해 고객과 제품의 상호작용을 자동화.

## 03-3. OAK-D 모듈
- **OAK-D 모듈 소개**: OAK-D(OpenCV AI Kit Depth)는 AI 및 컴퓨터 비전 기능을 갖춘 카메라 모듈로, 실시간으로 깊이 정보를 제공하며 다양한 응용 분야에 적합합니다.

- **주요 특징**:
  1. **내장형 AI**: Myriad X VPU를 탑재하여 온보드 AI 처리 가능.
  2. **스테레오 비전**: 두 개의 모노 카메라로 깊이 정보 계산.
  3. **컬러 카메라**: 고해상도 컬러 카메라로 세부 정보를 캡처.
  4. **플러그 앤 플레이**: USB를 통해 쉽게 연결 및 사용 가능.
  5. **다양한 SDK 지원**: DepthAI 및 OpenCV와 같은 SDK 지원.

- **응용 사례**:
  - 로봇 내비게이션
  - 객체 추적 및 감지
  - 증강 현실 및 가상 현실 애플리케이션

### OAK-D PRO의 자체 기능과 기술
- **IMU 센서**: 내장된 IMU 센서로 자세와 움직임을 추적할 수 있다.
- **AI 프로세싱**: 내장된 AI 칩으로 머신러닝 모델을 로컬에서 실행 가능.
- **다중 카메라**: 컬러 카메라와 두 개의 모노 카메라를 사용하여 스테레오 비전 지원.
- **ROS 지원**: 로봇 운영 체제(ROS)와의 통합 용이.

---

# 간단한 예제 실습

## Jetson Orin Nano에서 Camera module setting (Connection)

### 준비물
- Jetson Orin Nano
- OAK-D PRO Camera
- USB-C 케이블

### 필수 라이브러리 설치


### 단계
1. Jetson Orin Nano와 OAK-D PRO를 USB-C 케이블로 연결합니다.
2. Jetson Orin Nano의 터미널을 열고, `lsusb` 명령어로 카메라가 연결되었는지 확인합니다.
3. DepthAI Python 패키지를 설치합니다:
   ```bash
   pip install depthai
   ```
4. DepthAI 샘플 코드를 실행하여 연결 테스트:
   ```python
   import depthai as dai

   pipeline = dai.Pipeline()

   cam_rgb = pipeline.createColorCamera()
   cam_rgb.setPreviewSize(640, 480)
   cam_rgb.setInterleaved(False)
   cam_rgb.setFps(30)

   xout_preview = pipeline.createXLinkOut()
   xout_preview.setStreamName("rgb")

   cam_rgb.preview.link(xout_preview.input)

   with dai.Device(pipeline) as device:
       queue = device.getOutputQueue(name="rgb", maxSize=4, blocking=False)
       while True:
           frame = queue.get().getCvFrame()
           cv2.imshow("rgb", frame)
           if cv2.waitKey(1) == ord('q'):
               break
   ```

## Jetson Orin Nano에서 DeepStream + OAK-D PRO

### 설치 및 구성
1. DeepStream 설치:
   ```bash
   sudo apt-get install deepstream-6.1
   ```
2. OAK-D PRO와 DeepStream을 연결하기 위한 플러그인 설치:
   ```bash
   git clone https://github.com/NVIDIA-AI-IOT/deepstream_plugins
   cd deepstream_plugins
   make
   sudo make install
   ```

3. 기본 파이프라인 구성:
   - RTSP, USB 카메라, 파일 입력 모두 지원.

## TEST 3: 다중 파이프라인 구성 (RTSP, File, Camera, ...)

### 파이프라인 예제
1. **RTSP 스트림**:
   ```bash
   gst-launch-1.0 rtspsrc location=rtsp://<ip-address>:<port>/stream ! decodebin ! videoconvert ! autovideosink
   ```
2. **파일 스트림**:
   ```bash
   gst-launch-1.0 filesrc location=/path/to/video.mp4 ! decodebin ! videoconvert ! autovideosink
   ```
3. **USB 카메라 스트림**:
   ```bash
   gst-launch-1.0 v4l2src device=/dev/video0 ! decodebin ! videoconvert ! autovideosink
   ```

### TEST1-USBCAM : pipeline with USB camera input

```python
import gi
import cv2
import depthai as dai

gi.require_version('Gst', '1.0')
from gi.repository import Gst

# Initialize GStreamer
Gst.init(None)

pipeline = dai.Pipeline()

cam_rgb = pipeline.createColorCamera()
cam_rgb.setPreviewSize(640, 480)
cam_rgb.setInterleaved(False)
cam_rgb.setFps(30)

xout_preview = pipeline.createXLinkOut()
xout_preview.setStreamName("rgb")

cam_rgb.preview.link(xout_preview.input)

with dai.Device(pipeline) as device:
    queue = device.getOutputQueue(name="rgb", maxSize=4, blocking=False)
    while True:
        frame = queue.get().getCvFrame()
        cv2.imshow("rgb", frame)
        if cv2.waitKey(1) == ord('q'):
            break
```
